In [1]:
import fortuna
import numpy as np
import os
import pickle
import itertools
home_path = os.getcwd()

# Dueling bandits
https://reader.elsevier.com/reader/sd/pii/S0022000012000281?token=B68D38520003E31E4AAD0A97335C48912D85A7B5E3835743F296EB79DCCDD8DA3F9F6543777EA6D15BCE4820C421CE77


#### Interleaved filter 2



### Notes
* can we use the dG to improve the rate of pruning
* can we incorporate ddG into the CI metric?
* could this be done as a batch process more efficiently for the case of many bandits? Cluster molecules following #heavy atom difference or another metric correlated to the thermodynamic length of the process into select groups, and then compare the 'winners' of each batch?
* Can we initiate the P and c when we change the lead bandit in an intelligent way, such as to 'hot start' it

In [2]:
class duellingBernoulli(fortuna.Bernoulli):
    def __init__(self,p):
        self.P = None
        self.c = None
        self.d = None
        self.won = 0
        super(duellingBernoulli,self).__init__(p)
        
    def calcP(self):
        self.P =  (self.won) / self.steps
        
    def calcc(self,t):
        # two different equations for this in different versions of the manuscript
        #not sure which is correct
        self.c = (np.log(1/d)/t)**0.5 
        
    def setd(self,d):
        self.d = d
    
    def update(self,reward,t):
        # unsure if the t here is the overall t, or the T from this round.
        self.steps += 1
        self.won += reward
        self.calcP()
        self.calcc(t)
        
    def check_in_CI(self):
#         print(f'bounds: {(self.P - self.c)} -> {(self.P + self.c)}')
        if 0.5 > (self.P - self.c) and 0.5 < (self.P + self.c):
            return True
        else:
            return False
        
    def reset(self):
        self.steps = 0
        self.P = None 
        self.c = None
        self.d = None
        self.won = 0        
        
        
class comparison(object):
    def __init__(self,pair,dg,ddg):
        self.pair = pair
        self.dg = dg
        self.ddg = ddg
        self.i = 0
        
    def get_dg(self):
        # need a safety net so that it can't go past the end of the array
        result = self.dg[self.i]
        self.i += 1
        return result

In [3]:
# this needs a fully connected map to properly analyse
os.chdir('/Users/brucemah/bin/perses/examples/perses-testing/repex')

everything = []

for sim in os.listdir():
    filename = f'{sim}/results.pi'
    if os.path.isfile(filename):
        print(filename)
        out = pickle.load(open(filename,'rb'))
        everything.append(out)


lig5to2/results.pi
lig1to4/results.pi
lig1to3/results.pi
lig1to5/results.pi
lig5to3/results.pi
lig5to4/results.pi
lig0to2/results.pi
lig0to5/results.pi
lig4to3/results.pi
lig4to5/results.pi
lig4to2/results.pi
lig0to4/results.pi
lig0to3/results.pi
lig3to6/results.pi
lig3to0/results.pi
lig6to1/results.pi
lig2to0/results.pi
lig2to6/results.pi
lig2to1/results.pi
lig6to0/results.pi
lig5to1/results.pi
lig5to6/results.pi
lig1to0/results.pi
lig1to6/results.pi
lig5to0/results.pi
lig0to6/results.pi
lig0to1/results.pi
lig4to0/results.pi
lig4to1/results.pi
lig4to6/results.pi
lig3to2/results.pi
lig3to5/results.pi
lig3to4/results.pi
lig6to5/results.pi
lig6to2/results.pi
lig2to3/results.pi
lig2to5/results.pi
lig6to3/results.pi
lig6to4/results.pi


In [4]:
K = 7
bandits = [duellingBernoulli(None) for i in range(K)]

B_hat_id = np.random.choice(K)
B_hat = bandits[B_hat_id]
print(B_hat_id)

all_comparisons = []

for i,j in itertools.combinations(range(K),2):
    filename = f'lig{i}to{j}/results.pi'
    if os.path.isfile(filename):
        out = pickle.load(open(filename,'rb'))
        edge = comparison((i,j),out['dg'],out['ddg'])
        all_comparisons.append(edge)
    elif os.path.isfile(f'lig{j}to{i}/results.pi'):
        out = pickle.load(open(f'lig{j}to{i}/results.pi','rb'))
        fixed = [-x for x in out['dg']]
        edge = comparison((i,j),fixed,out['ddg'])
        all_comparisons.append(edge)
    else:
        print('Nothing to use')

0


In [5]:
print([(x.pair,len(x.dg)) for x in all_comparisons])

[((0, 1), 100), ((0, 2), 100), ((0, 3), 100), ((0, 4), 100), ((0, 5), 100), ((0, 6), 100), ((1, 2), 100), ((1, 3), 100), ((1, 4), 100), ((1, 5), 100), ((1, 6), 100), ((2, 3), 100), ((2, 4), 100), ((2, 5), 100), ((2, 6), 100), ((3, 4), 100), ((3, 5), 100), ((3, 6), 100), ((4, 5), 100), ((4, 6), 100), ((5, 6), 100)]


In [6]:
W = [b for b in bandits if b is not B_hat]

T = 100 # total comparisons
T_run = 0 # comparisons so far

d = 1. / (T * K**2)

won = False

for bandit in bandits:
    bandit.setd(d)


while T_run < T:
    for bandit in W:
        a = B_hat_id
        b = bandits.index(bandit)
        comparison = [x for x in all_comparisons if x.pair == (a,b)]
        if len(comparison) == 1:
            result = comparison[0].get_dg()
            if result < 0.:
                reward = True
            else:
                reward = False
        else:
            comparison = [x for x in all_comparisons if x.pair == (b,a)]
            if len(comparison) == 1:
                result = comparison[0].get_dg()
                if result > 0.:
                    reward = True
                else:
                    reward = False

        
        T_run += 1
        bandit.update(reward,T_run)


    # pruning
    for bandit in W:
        if bandit.P > 0.5 and bandit.check_in_CI() == False:
            print(f'Bandit {bandits.index(bandit)} is out. There are now {len(W)} bandits remaining')
            W = [b for b in W if b is not bandit] # removing this bandit from the list


    for bandit in W:
        if bandit.P < 0.5 and bandit.check_in_CI() == False:
            # we have a winning bandit
            B_hat = bandit
            B_hat_id = bandits.index(bandit)
            W = [b for b in W if b is not B_hat]            
            print(f'The bandit has been beaten. The main bandit is now {B_hat_id}', 
                  f'and there are now {len(W)+1} remaining bandits')
            won = True
            
    if len(W) == 0:
        print(f'FINISHED after {T_run} steps')
        T_final = int(T_run)
        T_run = 2*T
        
    if won:
        for bandit in W:
            bandit.reset()
            won = False
print('reached the end')

Bandit 5 is out. There are now 6 bandits remaining
The bandit has been beaten. The main bandit is now 6 and there are now 5 remaining bandits
Bandit 2 is out. There are now 4 bandits remaining
Bandit 3 is out. There are now 3 bandits remaining
Bandit 4 is out. There are now 2 bandits remaining
The bandit has been beaten. The main bandit is now 1 and there are now 1 remaining bandits
FINISHED after 40 steps
reached the end


In [7]:
print(f'The winning bandit is: {B_hat_id}')

The winning bandit is: 1
